In [1]:
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [2]:
import os
from collections import defaultdict
from random import shuffle

import xml.etree.ElementTree as ET
from tqdm import tqdm
import pandas as pd
import pybboxes as pbx
import shutil

In [3]:
annotations_path = "../data/annotations/"
images_path = "../data/images/"

In [4]:
annotations = os.listdir(annotations_path)
images = os.listdir(images_path)

In [5]:
img_name_list = []
width_list = []
height_list = []
label_list = []
x_min = []
y_min = []
x_max = []
y_max = []

In [6]:
for i in tqdm(range(len(annotations))):
    tree = ET.parse(os.path.join(annotations_path, annotations[i]))
    root = tree.getroot()
    img_name = root.find("filename").text

    size = root.find("size")
    width = int(size.find("width").text)
    height = int(size.find("height").text)

    for group in root.findall("object"):
        label = group.find("name").text
        bbox = group.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        ymin = int(bbox.find("ymin").text)
        xmax = int(bbox.find("xmax").text)
        ymax = int(bbox.find("ymax").text)

        img_name_list.append(img_name)
        width_list.append(width)
        height_list.append(height)
        x_min.append(xmin)
        y_min.append(ymin)
        x_max.append(xmax)
        y_max.append(ymax)
        label_list.append(label)

100%|██████████| 877/877 [00:08<00:00, 109.05it/s]


In [7]:
dataset = pd.DataFrame(
    {
        "img_name": img_name_list,
        "width": width_list,
        "height": height_list,
        "x_min": x_min,
        "y_min": y_min,
        "x_max": x_max,
        "y_max": y_max,
        "label": label_list,
    }
)
dataset.head()

,img_name,width,height,x_min,y_min,x_max,y_max,label
0,road0.png,267,400,98,62,208,232,trafficlight
1,road1.png,400,283,154,63,258,281,trafficlight
2,road10.png,400,267,106,3,244,263,trafficlight
3,road100.png,400,385,35,5,363,326,speedlimit
4,road101.png,400,200,195,7,392,194,speedlimit


In [8]:
classes = dataset["label"].unique().tolist()
class_training = {idx: label for idx, label in enumerate(classes)}
classes = {label: idx for idx, label in enumerate(classes)}
classes

{'trafficlight': 0, 'speedlimit': 1, 'crosswalk': 2, 'stop': 3}

In [9]:
dataset["class"] = dataset["label"].map(classes)

In [10]:
dataset.head()

,img_name,width,height,x_min,y_min,x_max,y_max,label,class
0,road0.png,267,400,98,62,208,232,trafficlight,0
1,road1.png,400,283,154,63,258,281,trafficlight,0
2,road10.png,400,267,106,3,244,263,trafficlight,0
3,road100.png,400,385,35,5,363,326,speedlimit,1
4,road101.png,400,200,195,7,392,194,speedlimit,1


In [11]:
imgs = defaultdict(list)
for idx, dt in tqdm(dataset.iterrows()):
    sample_label_list = []
    img_name = dt["img_name"]
    xmin = int(dt["x_min"])
    ymin = int(dt["y_min"])
    xmax = int(dt["x_max"])
    ymax = int(dt["y_max"])
    class_num = dt["class"]
    W, H = int(dt["width"]), int(dt["height"])

    voc_bbox = (int(xmin), int(ymin), int(xmax), int(ymax))

    x_center, y_center, w, h = pbx.convert_bbox(
        voc_bbox, from_type="voc", to_type="yolo", image_size=(W, H)
    )

    sample_label_list.append(str(class_num))
    sample_label_list.append(str(x_center))
    sample_label_list.append(str(y_center))
    sample_label_list.append(str(w))
    sample_label_list.append(str(h))
    line = " ".join(sample_label_list)

    imgs[img_name].append(line)

1244it [00:00, 9163.59it/s]


In [12]:
labels_dir = "../data/labels"
if os.path.exists(labels_dir):
    shutil.rmtree(labels_dir)
os.mkdir(labels_dir)

In [13]:
for img_name, lines in imgs.items():
    img_name = img_name.split('.')[0]
    with open(f'{labels_dir}/{img_name}.txt', 'w') as f:
        for line in lines:
            f.write(line)
            f.write('\n')

In [14]:
train_dir = "../data/train/"
val_dir = "../data/val"
labels_path = "../data/labels/"

In [15]:
if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

if os.path.exists(val_dir):
    shutil.rmtree(val_dir)

os.mkdir(train_dir)
os.mkdir(val_dir)

# train, val each containing images and labels folders
os.mkdir(train_dir + "/images")
os.mkdir(train_dir + "/labels")
os.mkdir(val_dir + "/images")
os.mkdir(val_dir + "/labels")

In [ ]:
files = os.listdir(images_path)
shuffle(files)

In [ ]:
def split(files, ratio):
    elements = len(files)
    middle = int(elements * ratio)
    return [files[:middle], files[middle:]]

def copy_files(images_path, labels_path, destination_path, files):
    
    for file_name in files:
        file_name = file_name.split('.')[0]

        src = images_path + f'/{file_name}.png'
        dst = destination_path + '/images'
        shutil.copy(src, dst)

        src = labels_path + f'/{file_name}.txt'
        dst = destination_path + '/labels'
        shutil.copy(src, dst)

In [ ]:
train_ratio = 0.75
train_files, val_files = split(files, train_ratio)

root = 'data/traffic_sign_data'

copy_files(images_path, labels_path, train_dir, train_files)
copy_files(images_path, labels_path, val_dir, val_files)

In [ ]:
with open(f'../data/sign_data.yaml', 'w') as f:
    f.write('train: ../../data/train/images\n')
    f.write('val: ../../data/val/images\n')
    f.write('nc: 4\n')
    f.write(f"names: {class_training}")

In [ ]:
!python yolov5/train.py --img 320 --batch 64 --epochs 20 --data ../data/sign_data.yaml --weights yolov5s.pt


Because of some problems My PC can't show the progress of the command above then I tried to run it with command prompt and the result are shown below

C:\Users\ADMIN\OneDrive - EPITA\Project-Demo\road-sign-detection\notebooks>python yolov5/train.py --img 320 --batch 64 --epochs 20 --data ../data/sign_data.yaml --weights yolov5s.pt
train: weights=yolov5s.pt, cfg=, data=../data/sign_data.yaml, hyp=yolov5\data\hyps\hyp.scratch-low.yaml, epochs=20, batch_size=64, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5\runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5
unknown option: -
usage: git [-v | --version] [-h | --help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           [--super-prefix=<path>] [--config-env=<name>=<envvar>]
           <command> [<args>]
YOLOv5  2023-12-4 Python-3.9.13 torch-2.1.0+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0
Comet: run 'pip install comet_ml' to automatically track and visualize YOLOv5  runs in Comet
TensorBoard: Start with 'tensorboard --logdir yolov5\runs\train', view at http://localhost:6006/
Overriding model.yaml nc=80 with nc=4

                 from  n    params  module                                  arguments
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]
  2                -1  1     18816  models.common.C3                        [64, 64, 1]
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]
  4                -1  2    115712  models.common.C3                        [128, 128, 2]
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]
  6                -1  3    625152  models.common.C3                        [256, 256, 3]
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]
  8                -1  1   1182720  models.common.C3                        [512, 512, 1]
  9                -1  1    656896  models.common.SPPF                      [512, 512, 5]
 10                -1  1    131584  models.common.Conv                      [512, 256, 1, 1]
 11                -1  1         0  torch.nn.modules.upsampling.Upsample    [None, 2, 'nearest']
 12           [-1, 6]  1         0  models.common.Concat                    [1]
 13                -1  1    361984  models.common.C3                        [512, 256, 1, False]
 14                -1  1     33024  models.common.Conv                      [256, 128, 1, 1]
 15                -1  1         0  torch.nn.modules.upsampling.Upsample    [None, 2, 'nearest']
 16           [-1, 4]  1         0  models.common.Concat                    [1]
 17                -1  1     90880  models.common.C3                        [256, 128, 1, False]
 18                -1  1    147712  models.common.Conv                      [128, 128, 3, 2]
 19          [-1, 14]  1         0  models.common.Concat                    [1]
 20                -1  1    296448  models.common.C3                        [256, 256, 1, False]
 21                -1  1    590336  models.common.Conv                      [256, 256, 3, 2]
 22          [-1, 10]  1         0  models.common.Concat                    [1]
 23                -1  1   1182720  models.common.C3                        [512, 512, 1, False]
 24      [17, 20, 23]  1     24273  models.yolo.Detect                      [4, [[10, 13, 16, 30, 33, 23], [30, 61, 62, 45, 59, 119], [116, 90, 156, 198, 373, 326]], [128, 256, 512]]
Model summary: 214 layers, 7030417 parameters, 7030417 gradients, 16.0 GFLOPs

Transferred 343/349 items from yolov5s.pt
optimizer: SGD(lr=0.01) with parameter groups 57 weight(decay=0.0), 60 weight(decay=0.0005), 60 bias
train: Scanning C:\Users\ADMIN\OneDrive - EPITA\Project-Demo\road-sign-detection\data\train\labels.cache... 657 images,
val: Scanning C:\Users\ADMIN\OneDrive - EPITA\Project-Demo\road-sign-detection\data\val\labels.cache... 220 images, 0 b

AutoAnchor: 5.45 anchors/target, 1.000 Best Possible Recall (BPR). Current anchors are a good fit to dataset
Plotting labels to yolov5\runs\train\exp3\labels.jpg...
Image sizes 320 train, 320 val
Using 8 dataloader workers
Logging results to yolov5\runs\train\exp3
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       0/19         0G     0.1228    0.01454    0.04879         46        320: 100%|██████████| 11/11 [01:05<00:00,  5.
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:09<0
                   all        220        317     0.0013       0.31    0.00295   0.000718

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       1/19         0G     0.1023    0.01854    0.03359         41        320: 100%|██████████| 11/11 [00:53<00:00,  4.
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:08<0
                   all        220        317      0.821      0.102      0.106     0.0267

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       2/19         0G    0.08426    0.02225    0.02589         55        320: 100%|██████████| 11/11 [00:53<00:00,  4.
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:07<00:00,  3.95s/it]
                   all        220        317      0.709      0.143     0.0989     0.0235

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       3/19         0G    0.07246    0.02272     0.0235         58        320: 100%|██████████| 11/11 [00:52<00:00,  4.75s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:07<00:00,  3.83s/it]
                   all        220        317      0.445      0.271       0.16     0.0491

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       4/19         0G    0.06494    0.02001    0.01967         50        320: 100%|██████████| 11/11 [00:52<00:00,  4.79s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:07<00:00,  3.57s/it]
                   all        220        317      0.394      0.334      0.239     0.0953

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       5/19         0G    0.06057    0.01808    0.01563         60        320: 100%|██████████| 11/11 [00:52<00:00,  4.81s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:07<00:00,  3.63s/it]
                   all        220        317      0.756      0.305      0.272      0.111

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       6/19         0G    0.05812    0.01621    0.01368         37        320: 100%|██████████| 11/11 [00:53<00:00,  4.89s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:06<00:00,  3.45s/it]
                   all        220        317      0.806      0.322      0.428      0.207

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       7/19         0G    0.05587     0.0161    0.01223         51        320: 100%|██████████| 11/11 [00:53<00:00,  4.87s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:06<00:00,  3.36s/it]
                   all        220        317      0.411      0.638      0.395      0.184

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       8/19         0G    0.05098    0.01385   0.009572         44        320: 100%|██████████| 11/11 [00:52<00:00,  4.80s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:06<00:00,  3.42s/it]
                   all        220        317      0.632      0.666      0.607      0.286

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       9/19         0G    0.04735    0.01305   0.007595         39        320: 100%|██████████| 11/11 [00:53<00:00,  4.84s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:06<00:00,  3.43s/it]
                   all        220        317      0.702      0.619      0.656      0.301

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      10/19         0G    0.04416    0.01173   0.006267         48        320: 100%|██████████| 11/11 [00:55<00:00,  5.03s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:06<00:00,  3.29s/it]
                   all        220        317      0.599       0.69      0.656      0.337

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      11/19         0G    0.04196    0.01132   0.005493         42        320: 100%|██████████| 11/11 [00:55<00:00,  5.03s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:06<00:00,  3.18s/it]
                   all        220        317      0.843      0.687      0.763      0.439

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      12/19         0G    0.03893    0.01087   0.005054         51        320: 100%|██████████| 11/11 [00:54<00:00,  5.00s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:06<00:00,  3.23s/it]
                   all        220        317      0.694      0.701      0.704      0.405

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      13/19         0G    0.03649    0.01051   0.004301         28        320: 100%|██████████| 11/11 [00:54<00:00,  4.99s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:06<00:00,  3.21s/it]
                   all        220        317      0.815      0.755      0.794      0.479

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      14/19         0G    0.03262       0.01   0.004096         33        320: 100%|██████████| 11/11 [00:53<00:00,  4.86s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:05<00:00,  2.95s/it]
                   all        220        317      0.805      0.757      0.802      0.478

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      15/19         0G    0.03042   0.009585   0.003447         37        320: 100%|██████████| 11/11 [00:53<00:00,  4.83s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:05<00:00,  2.91s/it]
                   all        220        317      0.882      0.787      0.847      0.542

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      16/19         0G     0.0282   0.009109   0.003347         41        320: 100%|██████████| 11/11 [00:53<00:00,  4.87s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:06<00:00,  3.23s/it]
                   all        220        317      0.933      0.775      0.866      0.559

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      17/19         0G    0.02676   0.008945   0.003462         39        320: 100%|██████████| 11/11 [00:52<00:00,  4.73s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:05<00:00,  2.91s/it]
                   all        220        317      0.882      0.805      0.866      0.572

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      18/19         0G    0.02412   0.008744   0.002964         41        320: 100%|██████████| 11/11 [00:52<00:00,  4.79s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:05<00:00,  2.90s/it]
                   all        220        317       0.91      0.796      0.873      0.596

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      19/19         0G    0.02276   0.008834    0.00272         34        320: 100%|██████████| 11/11 [00:52<00:00,  4.78s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:05<00:00,  2.96s/it]
                   all        220        317      0.928      0.817      0.887      0.608

20 epochs completed in 0.341 hours.
Optimizer stripped from yolov5\runs\train\exp3\weights\last.pt, 14.3MB
Optimizer stripped from yolov5\runs\train\exp3\weights\best.pt, 14.3MB

Validating yolov5\runs\train\exp3\weights\best.pt...
Fusing layers...
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 2/2 [00:06<00:00,  3.15s/it]
                   all        220        317      0.927      0.817      0.887      0.608
          trafficlight        220         42      0.837      0.786      0.846      0.451
            speedlimit        220        203      0.946       0.98      0.988      0.806
             crosswalk        220         55      0.925      0.677      0.788      0.488
                  stop        220         17          1      0.824      0.927      0.686
Results saved to yolov5\runs\train\exp3